In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

**7. In Sections 5.3.2 and 5.3.3, we saw that the cv.glm() function can be
used in order to compute the LOOCV test error estimate. Alterna-
tively, one could compute those quantities using just the glm() and
200 predict.glm() functions, and a for loop. You will now take this ap-
proach in order to compute the LOOCV error for a simple logistic
regression model on the Weekly data set. Recall that in the context
of classification problems, the LOOCV error is given in (5.4).**

In [3]:
Weekly = pd.read_csv("data/Weekly.csv").drop('Unnamed: 0',axis = 1)
Weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [4]:
#将Weekly哑变量便于分析
key_title = 'Direction'
dummies = pd.get_dummies(Weekly['Direction'],prefix = key_title)
Weekly_Dummy = Weekly.join(dummies)
Weekly_Dummy.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction,Direction_Down,Direction_Up
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down,1,0
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down,1,0
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up,0,1
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up,0,1
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up,0,1


In [5]:
glm_smf = smf.logit('Direction_Up ~ Lag1 + Lag2',data = Weekly_Dummy).fit()
glm_smf.summary()

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Direction_Up   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1086
Method:                           MLE   Df Model:                            2
Date:                Tue, 31 Dec 2019   Pseudo R-squ.:                0.005335
Time:                        16:58:18   Log-Likelihood:                -744.11
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                   0.01848
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2212      0.061      3.599      0.000       0.101       0.342
Lag1          -0.0387      0.026     -1.477      0.140      -0.090       0.013
Lag2           0.0602      0.027      2.270      0.023       0.008       0.112
==============================================================================
"""

**(b) Fit a logistic regression model that predicts Direction using Lag1
and Lag2 using all but the first observation.**

In [6]:
glm_smf = smf.logit('Direction_Up ~ Lag1 + Lag2',data = Weekly_Dummy.iloc[1:]).fit()
glm_smf.summary()

Optimization terminated successfully.
         Current function value: 0.683147
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Direction_Up   No. Observations:                 1088
Model:                          Logit   Df Residuals:                     1085
Method:                           MLE   Df Model:                            2
Date:                Tue, 31 Dec 2019   Pseudo R-squ.:                0.005387
Time:                        16:58:18   Log-Likelihood:                -743.26
converged:                       True   LL-Null:                       -747.29
Covariance Type:            nonrobust   LLR p-value:                   0.01785
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2232      0.061      3.630      0.000       0.103       0.344
Lag1          -0.0384      0.026     -1.466      0.143      -0.090       0.013
Lag2           0.0608      0.027      2.291      0.022       0.009       0.113
==============================================================================
"""

**(c) Use the model from (b) to predict the direction of the first obser-
vation. You can do this by predicting that the first observation
will go up if P( Direction="Up" | Lag1 , Lag2 ) > 0.5. Was this ob-
servation correctly classified?**

In [7]:
#这里不能使用Weekly_Dummy.iloc[0]，一个是Series，一个是DataFrame
glm_smf.predict(Weekly_Dummy.iloc[0:1]) > 0.5

0    True
dtype: bool

In [8]:
(glm_smf.predict(Weekly_Dummy.iloc[0:1]) > 0.5 ) == Weekly_Dummy.iloc[0:1]['Direction_Up']

0    False
dtype: bool

**(d) Write a for loop from i = 1 to i = n, where n is the number of
observations in the data set, that performs each of the following
steps:
i. Fit a logistic regression model using all but the ith obser-
vation to predict Direction using Lag1 and Lag2 .
ii. Compute the posterior probability of the market moving up
for the ith observation.
iii. Use the posterior probability for the ith observation in order
to predict whether or not the market moves up.
iv. Determine whether or not an error was made in predicting
the direction for the ith observation. If an error was made,
then indicate this as a 1, and otherwise indicate it as a 0.**

In [9]:
loocv_predict = []
loocv_result = []
for i in range(len(Weekly_Dummy)):
    glm_smf = smf.logit('Direction_Up ~ Lag1 + Lag2',data = Weekly_Dummy.drop(i)).fit()
    loocv_predict.append((glm_smf.predict(Weekly_Dummy.iloc[i:i + 1]) > 0.5)[i] * 1)
    loocv_result.append(((glm_smf.predict(Weekly_Dummy.iloc[i:i + 1]) > 0.5) != Weekly_Dummy.iloc[i:i + 1]['Direction_Up'])[i] * 1)
Weekly_Loocv = Weekly_Dummy[['Year','Direction_Up']].copy()
Weekly_Loocv['Direction_predict'] = loocv_predict
Weekly_Loocv['Loocv_result'] = loocv_result    

Optimization terminated successfully.
         Current function value: 0.683147
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683149
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683416
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683253
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683454
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683181
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683432
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683335
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683402
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683203
  

Optimization terminated successfully.
         Current function value: 0.683366
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683244
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683134
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683389
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683352
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683234
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683480
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683129
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683113
  

Optimization terminated successfully.
         Current function value: 0.683346
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683208
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683460
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683302
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683167
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683405
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683344
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683155
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683149
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683168
  

Optimization terminated successfully.
         Current function value: 0.683464
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683252
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683176
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683409
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683243
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683410
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683302
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683100
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683391
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683376
  

Optimization terminated successfully.
         Current function value: 0.683127
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683438
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683233
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683416
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683353
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683187
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683122
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683205
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683209
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683362
  

...

Optimization terminated successfully.
         Current function value: 0.683198
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683380
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683338
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683439
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683199
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683138
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683183
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683166
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683182
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683194
  

Optimization terminated successfully.
         Current function value: 0.683411
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683291
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683087
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683419
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683272
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683448
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683341
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683363
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683133
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683171
  

Optimization terminated successfully.
         Current function value: 0.683240
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683098
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683396
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683312
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683390
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683448
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683352
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683121
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683448
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683333
  

Optimization terminated successfully.
         Current function value: 0.682939
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683581
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683320
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683475
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683435
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683158
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683426
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683352
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683251
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683573
  

Optimization terminated successfully.
         Current function value: 0.683175
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683435
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683351
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683408
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683397
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683397
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683247
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683496
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683245
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683398
  

In [10]:
Weekly_Loocv.head()

,Year,Direction_Up,Direction_predict,Loocv_result
0,1990,0,1,1
1,1990,0,1,1
2,1990,1,1,0
3,1990,1,0,1
4,1990,1,1,0


In [11]:
Weekly_Loocv.describe()

,Year,Direction_Up,Direction_predict,Loocv_result
count,1089.000000,1089.000000,1089.000000,1089.000000
mean,2000.048669,0.555556,0.932048,0.449954
std,6.033182,0.497132,0.251780,0.497718
min,1990.000000,0.000000,0.000000,0.000000
25%,1995.000000,0.000000,1.000000,0.000000
50%,2000.000000,1.000000,1.000000,0.000000
75%,2005.000000,1.000000,1.000000,1.000000
max,2010.000000,1.000000,1.000000,1.000000


In [12]:
Weekly_Loocv['Loocv_result'].sum()

490

**(e) Take the average of the n numbers obtained in (d)iv in order to
obtain the LOOCV estimate for the test error. Comment on the
results.**

In [13]:
Weekly_Loocv['Loocv_result'].mean()

0.44995408631772266

和第四章实验4.6.2直接用所有数据做训练数据算出的训练错误率差不多。